In [1]:
!pip install av
! wget https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
# Download HMDB51 data and splits from serre lab website
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
! wget http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/test_train_splits.rar

     |████████████████████████████████| 37.2 MB 28 kB/s 
--2021-07-30 12:32:33--  https://raw.githubusercontent.com/pytorch/vision/6de158c473b83cf43344a0651d7c01128c7850e6/references/video_classification/transforms.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3102 (3.0K) [text/plain]
Saving to: ‘transforms.py’

transforms.py       100%[===================>]   3.03K  --.-KB/s    in 0s      

2021-07-30 12:32:33 (48.4 MB/s) - ‘transforms.py’ saved [3102/3102]

--2021-07-30 12:32:34--  http://serre-lab.clps.brown.edu/wp-content/uploads/2013/10/hmdb51_org.rar
Resolving serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)... 128.148.254.114
Connecting to serre-lab.clps.brown.edu (serre-lab.clps.brown.edu)|128.148.254.114|:80... connected.
HTTP request sent, 

In [2]:
pip install git+https://github.com/Atze00/MoViNet-pytorch.git


  Cloning https://github.com/Atze00/MoViNet-pytorch.git (to revision v0.2) to /tmp/pip-req-build-1oz6sjw9
  Running command git clone -q https://github.com/Atze00/MoViNet-pytorch.git /tmp/pip-req-build-1oz6sjw9
  Running command git checkout -b v0.2 --track origin/v0.2
  Switched to a new branch 'v0.2'
  Branch 'v0.2' set up to track remote branch 'v0.2' from 'origin'.
     |████████████████████████████████| 49 kB 3.7 MB/s 
     |████████████████████████████████| 636 kB 13.3 MB/s 
  Created wheel for MoViNet-pytorch: filename=MoViNet_pytorch-0.2.0-py3-none-any.whl size=12025 sha256=daf5155db8c8b150e912021d5aa479ab2ef45bbd3aa5f9c8ae09a87ba9805367
  Stored in directory: /tmp/pip-ephem-wheel-cache-7vrz8xc3/wheels/59/99/a5/43c18d233e06a21774b6494a17871b37edf6d31308ae0fb822
  Created wheel for fvcore: filename=fvcore-0.1.5.post20210730-py3-none-any.whl size=60619 sha256=af28fe7c58e39f1f3bf2b06ab696ffe92e3233461dece85520c1993e31a6773c
  Stored in directory: /root/.cache/pip/wheels/fc/9c/12/9

In [3]:
# Extract and organize video data..
! mkdir -p video_data test_train_splits
! unrar e test_train_splits.rar test_train_splits
! rm test_train_splits.rar
! unrar e hmdb51_org.rar 
! rm hmdb51_org.rar
! mv *.rar video_data
import os
for files in os.listdir('video_data'):
    foldername = files.split('.')[0]
    os.system("mkdir -p video_data/" + foldername)
    os.system("unrar e video_data/"+ files + " video_data/"+foldername)

! rm video_data/*.rar



UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from test_train_splits.rar

Extracting  test_train_splits/brush_hair_test_split1.txt                   0%  OK 
Extracting  test_train_splits/brush_hair_test_split2.txt                   1%  OK 
Extracting  test_train_splits/brush_hair_test_split3.txt                   1%  OK 
Extracting  test_train_splits/cartwheel_test_split1.txt                    2%  OK 
Extracting  test_train_splits/cartwheel_test_split2.txt                    2%  OK 
Extracting  test_train_splits/cartwheel_test_split3.txt                    3%  OK 
Extracting  test_train_splits/catch_test_split1.txt                        4%  OK 
Extracting  test_train_splits/catch_test_split2.txt                        4%  OK 
Extracting  test_train_splits/catch_test_split3.txt                        5%  OK 
Extracting  test_train_splits/chew_test_split1.txt              

In [2]:
import time
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
import torch
import transforms as T
from movinets import MoViNet
from movinets.config import _C
from HMDB51 import HMDB51_to_ViolenceWrapper

torch.manual_seed(97)
num_frames = 16 # 16
clip_steps = 2
Bs_Train = 16
Bs_Test = 16

transform = transforms.Compose([  
                                                     
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((200, 200)),
                                 T.RandomHorizontalFlip(),
                                 #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.RandomCrop((172, 172))])
transform_test = transforms.Compose([                           
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((200, 200)),
                                 #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.CenterCrop((172, 172))])

hmdb51_train = HMDB51_to_ViolenceWrapper('video_data/', train=True, clip_steps=clip_steps, transform=transform, num_frames=num_frames)

hmdb51_test = HMDB51_to_ViolenceWrapper('video_data/', train=False, clip_steps=clip_steps, transform=transform_test, num_frames=num_frames)

train_loader = DataLoader(hmdb51_train, batch_size=Bs_Train, shuffle=True)
test_loader  = DataLoader(hmdb51_test, batch_size=Bs_Test, shuffle=False)

  0%|          | 0/423 [00:00<?, ?it/s]

100%|██████████| 423/423 [00:38<00:00, 10.90it/s]
/home/sangyoonyu/workspace/ADL/venv/lib/python3.10/site-packages/torchvision/datasets/video_utils.py:219: UserWarning: There aren't enough frames in the current video to get a clip for the given clip length and frames between clips. The video (and potentially others) will be skipped.
  warnings.warn(
100%|██████████| 423/423 [00:38<00:00, 10.89it/s]


In [3]:
def train_iter(model, optimz, data_load, loss_val):
    samples = len(data_load.dataset)
    model.train()
    model.cuda()
    model.clean_activation_buffers()
    optimz.zero_grad()
    for i, (data, target) in enumerate(data_load):
        out = F.log_softmax(model(data.cuda()), dim=1)
        loss = F.nll_loss(out, target.cuda())
        loss.backward()
        optimz.step()
        optimz.zero_grad()
        model.clean_activation_buffers()
        if i % 50 == 0:
            print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(samples) +
                  ' (' + '{:3.0f}'.format(100 * i / len(data_load)) + '%)]  Loss: ' +
                  '{:6.4f}'.format(loss.item()))
            loss_val.append(loss.item())
 
def evaluate(model, data_load, loss_val):
    model.eval()
    
    samples = len(data_load.dataset)
    csamp = 0
    tloss = 0
    model.clean_activation_buffers()
    with torch.no_grad():
        for data, target in data_load:
            output = F.log_softmax(model(data.cuda()), dim=1)
            loss = F.nll_loss(output, target.cuda(), reduction='sum')
            _, pred = torch.max(output, dim=1)
            
            tloss += loss.item()
            csamp += pred.eq(target.cuda()).sum()
            model.clean_activation_buffers()
    aloss = tloss / samples
    loss_val.append(aloss)
    print('\nAverage test loss: ' + '{:.4f}'.format(aloss) +
          '  Accuracy:' + '{:5}'.format(csamp) + '/' +
          '{:5}'.format(samples) + ' (' +
          '{:4.2f}'.format(100.0 * csamp / samples) + '%)\n')

In [9]:
model = MoViNet(_C.MODEL.MoViNetA3, causal = False, pretrained = True )

model.classifier[3] = torch.nn.Conv3d(2048, 2, (1,1,1))

model.cpu()

# load model from checkpoint
model.load_state_dict(torch.load('checkpoint/best.pt'))

model.cuda()

loss_val = []
evaluate(model, test_loader, loss_val)

/home/sangyoonyu/workspace/ADL/venv/lib/python3.10/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")



Average test loss: 2.6112  Accuracy:  696/ 2945 (23.63%)



In [5]:
N_EPOCHS = 2

# Full FT
model = MoViNet(_C.MODEL.MoViNetA3, causal = False, pretrained = True )
start_time = time.time()

trloss_val, tsloss_val = [], []
model.classifier[3] = torch.nn.Conv3d(2048, 2, (1,1,1))
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, train_loader, trloss_val)
    evaluate(model, test_loader, tsloss_val)
 
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1
[    0/ 7520 (  0%)]  Loss: 0.6920
[  800/ 7520 ( 11%)]  Loss: 0.3196
[ 1600/ 7520 ( 21%)]  Loss: 0.3332
[ 2400/ 7520 ( 32%)]  Loss: 0.2180
[ 3200/ 7520 ( 43%)]  Loss: 0.0633
[ 4000/ 7520 ( 53%)]  Loss: 0.1269
[ 4800/ 7520 ( 64%)]  Loss: 0.0231
[ 5600/ 7520 ( 74%)]  Loss: 0.0220
[ 6400/ 7520 ( 85%)]  Loss: 0.0281
[ 7200/ 7520 ( 96%)]  Loss: 0.0546

Average test loss: 0.1394  Accuracy: 2797/ 2945 (94.97%)

Epoch: 2
[    0/ 7520 (  0%)]  Loss: 0.0133
[  800/ 7520 ( 11%)]  Loss: 0.0362
[ 1600/ 7520 ( 21%)]  Loss: 0.0048
[ 2400/ 7520 ( 32%)]  Loss: 0.0034
[ 3200/ 7520 ( 43%)]  Loss: 0.0031
[ 4000/ 7520 ( 53%)]  Loss: 0.0034
[ 4800/ 7520 ( 64%)]  Loss: 0.0011
[ 5600/ 7520 ( 74%)]  Loss: 0.0031
[ 6400/ 7520 ( 85%)]  Loss: 0.0008
[ 7200/ 7520 ( 96%)]  Loss: 0.0026

Average test loss: 0.2044  Accuracy: 2784/ 2945 (94.53%)

Execution time: 1196.29 seconds


In [3]:
# Head FT
N_EPOCHS = 2

model = MoViNet(_C.MODEL.MoViNetA3, causal = False, pretrained = True )
start_time = time.time()

# freeze model except last layer
for param in model.parameters():
    param.requires_grad = False

model.classifier[3] = torch.nn.Conv3d(2048, 2, (1,1,1))

trloss_val, tsloss_val = [], []
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, train_loader, trloss_val)
    evaluate(model, test_loader, tsloss_val)
 
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1


/home/sangyoonyu/workspace/ADL/venv/lib/python3.10/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


[    0/ 7520 (  0%)]  Loss: 0.7021
[  800/ 7520 ( 11%)]  Loss: 0.6885
[ 1600/ 7520 ( 21%)]  Loss: 0.6712
[ 2400/ 7520 ( 32%)]  Loss: 0.6688
[ 3200/ 7520 ( 43%)]  Loss: 0.6784
[ 4000/ 7520 ( 53%)]  Loss: 0.6394
[ 4800/ 7520 ( 64%)]  Loss: 0.6139
[ 5600/ 7520 ( 74%)]  Loss: 0.6282
[ 6400/ 7520 ( 85%)]  Loss: 0.5781
[ 7200/ 7520 ( 96%)]  Loss: 0.6103

Average test loss: 0.6175  Accuracy: 2610/ 2945 (88.62%)

Epoch: 2
[    0/ 7520 (  0%)]  Loss: 0.6087
[  800/ 7520 ( 11%)]  Loss: 0.5688
[ 1600/ 7520 ( 21%)]  Loss: 0.6276
[ 2400/ 7520 ( 32%)]  Loss: 0.5743
[ 3200/ 7520 ( 43%)]  Loss: 0.5496
[ 4000/ 7520 ( 53%)]  Loss: 0.5873
[ 4800/ 7520 ( 64%)]  Loss: 0.5474
[ 5600/ 7520 ( 74%)]  Loss: 0.5366
[ 6400/ 7520 ( 85%)]  Loss: 0.5233
[ 7200/ 7520 ( 96%)]  Loss: 0.5644

Average test loss: 0.5526  Accuracy: 2716/ 2945 (92.22%)

Execution time: 1309.58 seconds


In [6]:
# Head MLP FT
N_EPOCHS = 2

model = MoViNet(_C.MODEL.MoViNetA3, causal = False, pretrained = True )
start_time = time.time()

# freeze model except classifier MLP layer
for param in model.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = True

model.classifier[3] = torch.nn.Conv3d(2048, 2, (1,1,1))

trloss_val, tsloss_val = [], []
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, train_loader, trloss_val)
    evaluate(model, test_loader, tsloss_val)
 
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1
[    0/ 7520 (  0%)]  Loss: 0.6927
[  800/ 7520 ( 11%)]  Loss: 0.5976
[ 1600/ 7520 ( 21%)]  Loss: 0.4098
[ 2400/ 7520 ( 32%)]  Loss: 0.3500
[ 3200/ 7520 ( 43%)]  Loss: 0.3029
[ 4000/ 7520 ( 53%)]  Loss: 0.2082
[ 4800/ 7520 ( 64%)]  Loss: 0.4062
[ 5600/ 7520 ( 74%)]  Loss: 0.2735
[ 6400/ 7520 ( 85%)]  Loss: 0.2531
[ 7200/ 7520 ( 96%)]  Loss: 0.2841

Average test loss: 0.2432  Accuracy: 2769/ 2945 (94.02%)

Epoch: 2
[    0/ 7520 (  0%)]  Loss: 0.2829
[  800/ 7520 ( 11%)]  Loss: 0.3504


In [4]:
# First and Head FT
N_EPOCHS = 2

model = MoViNet(_C.MODEL.MoViNetA3, causal = False, pretrained = True )
start_time = time.time()

# freeze model except first, last layer
for param in model.parameters():
    param.requires_grad = False

for param in model.conv1.parameters():
    param.requires_grad = True

model.classifier[3] = torch.nn.Conv3d(2048, 2, (1,1,1))

trloss_val, tsloss_val = [], []
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, train_loader, trloss_val)
    evaluate(model, test_loader, tsloss_val)
 
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1


OutOfMemoryError: CUDA out of memory. Tried to allocate 116.00 MiB. GPU 0 has a total capacty of 23.69 GiB of which 22.69 MiB is free. Including non-PyTorch memory, this process has 4.00 GiB memory in use. Process 1776135 has 2.14 GiB memory in use. Process 1784702 has 17.52 GiB memory in use. Of the allocated memory 3.56 GiB is allocated by PyTorch, and 132.07 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [13]:
# Conv1, Blocks and Head FT
N_EPOCHS = 2

model = MoViNet(_C.MODEL.MoViNetA3, causal = False, pretrained = True )
start_time = time.time()

# freeze model except conv1, blocks and last layer
for param in model.parameters():
    param.requires_grad = False

for param in model.conv1.parameters():
    param.requires_grad = True

for param in model.blocks.parameters():
    param.requires_grad = True

model.classifier[3] = torch.nn.Conv3d(2048, 51, (1,1,1))

trloss_val, tsloss_val = [], []
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, train_loader, trloss_val)
    evaluate(model, test_loader, tsloss_val)
 
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1
[    0/ 7520 (  0%)]  Loss: 3.9341
[  800/ 7520 ( 11%)]  Loss: 3.8683
[ 1600/ 7520 ( 21%)]  Loss: 3.7530
[ 2400/ 7520 ( 32%)]  Loss: 3.5150
[ 3200/ 7520 ( 43%)]  Loss: 3.2698
[ 4000/ 7520 ( 53%)]  Loss: 2.8049
[ 4800/ 7520 ( 64%)]  Loss: 2.2856
[ 5600/ 7520 ( 74%)]  Loss: 2.0012
[ 6400/ 7520 ( 85%)]  Loss: 2.2784
[ 7200/ 7520 ( 96%)]  Loss: 1.6320

Average test loss: 1.6970  Accuracy: 1934/ 2945 (65.67%)

Execution time: 537.16 seconds


In [4]:
# Blocks and Head FT
N_EPOCHS = 2

model = MoViNet(_C.MODEL.MoViNetA3, causal = False, pretrained = True )
start_time = time.time()

# freeze model except blocks and last layer
for param in model.parameters():
    param.requires_grad = False

for param in model.blocks.parameters():
    param.requires_grad = True

model.classifier[3] = torch.nn.Conv3d(2048, 2, (1,1,1))

trloss_val, tsloss_val = [], []
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, train_loader, trloss_val)
    evaluate(model, test_loader, tsloss_val)
 
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1


/home/sangyoonyu/workspace/ADL/venv/lib/python3.10/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


OutOfMemoryError: CUDA out of memory. Tried to allocate 204.00 MiB. GPU 0 has a total capacty of 23.69 GiB of which 99.62 MiB is free. Process 72943 has 18.09 GiB memory in use. Including non-PyTorch memory, this process has 5.49 GiB memory in use. Of the allocated memory 5.05 GiB is allocated by PyTorch, and 145.64 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [3]:
# Blocks, conv7 and head FT
N_EPOCHS = 2

model = MoViNet(_C.MODEL.MoViNetA3, causal = False, pretrained = True )
start_time = time.time()

# freeze conv1 layer
for param in model.conv1.parameters():
    param.requires_grad = False

for param in model.classifier.parameters():
    param.requires_grad = False

model.classifier[3] = torch.nn.Conv3d(2048, 2, (1,1,1))

trloss_val, tsloss_val = [], []
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, train_loader, trloss_val)
    evaluate(model, test_loader, tsloss_val)
 
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1


/home/sangyoonyu/workspace/ADL/venv/lib/python3.10/site-packages/torchvision/io/video.py:161: UserWarning: The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.
  warnings.warn("The pts_unit 'pts' gives wrong results. Please use pts_unit 'sec'.")


[    0/ 7520 (  0%)]  Loss: 0.7040
[  800/ 7520 ( 11%)]  Loss: 0.6532
[ 1600/ 7520 ( 21%)]  Loss: 0.5148
[ 2400/ 7520 ( 32%)]  Loss: 0.3706
[ 3200/ 7520 ( 43%)]  Loss: 0.3163
[ 4000/ 7520 ( 53%)]  Loss: 0.1021
[ 4800/ 7520 ( 64%)]  Loss: 0.0810
[ 5600/ 7520 ( 74%)]  Loss: 0.1422
[ 6400/ 7520 ( 85%)]  Loss: 0.0243
[ 7200/ 7520 ( 96%)]  Loss: 0.0126

Average test loss: 0.1233  Accuracy: 2830/ 2945 (96.10%)

Epoch: 2
[    0/ 7520 (  0%)]  Loss: 0.0148
[  800/ 7520 ( 11%)]  Loss: 0.0141
[ 1600/ 7520 ( 21%)]  Loss: 0.0338
[ 2400/ 7520 ( 32%)]  Loss: 0.0128
[ 3200/ 7520 ( 43%)]  Loss: 0.0102
[ 4000/ 7520 ( 53%)]  Loss: 0.0081
[ 4800/ 7520 ( 64%)]  Loss: 0.0040
[ 5600/ 7520 ( 74%)]  Loss: 0.0172
[ 6400/ 7520 ( 85%)]  Loss: 0.0033
[ 7200/ 7520 ( 96%)]  Loss: 0.0137

Average test loss: 0.1228  Accuracy: 2820/ 2945 (95.76%)

Execution time: 1105.53 seconds


In [7]:
# conv7 and mlp FT
N_EPOCHS = 2

model = MoViNet(_C.MODEL.MoViNetA3, causal = False, pretrained = True )
start_time = time.time()

# freeze conv1 layer
for param in model.conv1.parameters():
    param.requires_grad = False

for param in model.blocks.parameters():
    param.requires_grad = False

model.classifier[3] = torch.nn.Conv3d(2048, 2, (1,1,1))

trloss_val, tsloss_val = [], []
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, train_loader, trloss_val)
    evaluate(model, test_loader, tsloss_val)
 
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1
[    0/ 7520 (  0%)]  Loss: 0.6934
[  800/ 7520 ( 11%)]  Loss: 0.5516
[ 1600/ 7520 ( 21%)]  Loss: 0.4516
[ 2400/ 7520 ( 32%)]  Loss: 0.2586
[ 3200/ 7520 ( 43%)]  Loss: 0.2466
[ 4000/ 7520 ( 53%)]  Loss: 0.3309
[ 4800/ 7520 ( 64%)]  Loss: 0.4224
[ 5600/ 7520 ( 74%)]  Loss: 0.1148
[ 6400/ 7520 ( 85%)]  Loss: 0.4853
[ 7200/ 7520 ( 96%)]  Loss: 0.1583

Average test loss: 0.1920  Accuracy: 2789/ 2945 (94.70%)

Epoch: 2
[    0/ 7520 (  0%)]  Loss: 0.1737
[  800/ 7520 ( 11%)]  Loss: 0.1667
[ 1600/ 7520 ( 21%)]  Loss: 0.0488
[ 2400/ 7520 ( 32%)]  Loss: 0.0444
[ 3200/ 7520 ( 43%)]  Loss: 0.2826
[ 4000/ 7520 ( 53%)]  Loss: 0.1730
[ 4800/ 7520 ( 64%)]  Loss: 0.0406
[ 5600/ 7520 ( 74%)]  Loss: 0.0921
[ 6400/ 7520 ( 85%)]  Loss: 0.0414
[ 7200/ 7520 ( 96%)]  Loss: 0.0923

Average test loss: 0.1639  Accuracy: 2788/ 2945 (94.67%)

Execution time: 1058.32 seconds


In [16]:
# Blocks and Head FT
N_EPOCHS = 2

model = MoViNet(_C.MODEL.MoViNetA3, causal = False, pretrained = True )
start_time = time.time()

# freeze conv1, blocks layer
for param in model.conv1.parameters():
    param.requires_grad = False
    
for param in model.blocks.parameters():
    param.requires_grad = False

model.classifier[3] = torch.nn.Conv3d(2048, 51, (1,1,1))

trloss_val, tsloss_val = [], []
optimz = optim.Adam(model.parameters(), lr=0.00005)
for epoch in range(1, N_EPOCHS + 1):
    print('Epoch:', epoch)
    train_iter(model, optimz, train_loader, trloss_val)
    evaluate(model, test_loader, tsloss_val)
 
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

Epoch: 1
[    0/ 7520 (  0%)]  Loss: 3.9328
[  800/ 7520 ( 11%)]  Loss: 3.8582
[ 1600/ 7520 ( 21%)]  Loss: 3.6928
[ 2400/ 7520 ( 32%)]  Loss: 3.4207
[ 3200/ 7520 ( 43%)]  Loss: 3.0888
[ 4000/ 7520 ( 53%)]  Loss: 3.0386
[ 4800/ 7520 ( 64%)]  Loss: 2.3549
[ 5600/ 7520 ( 74%)]  Loss: 2.4311
[ 6400/ 7520 ( 85%)]  Loss: 2.3564
[ 7200/ 7520 ( 96%)]  Loss: 1.8804

Average test loss: 2.2315  Accuracy: 1756/ 2945 (59.63%)

Execution time: 509.30 seconds
